# SRE Agent

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_openai import ChatOpenAI
gpt5 = ChatOpenAI(model="gpt-5")

## Data structure

In [9]:
from typing import TypedDict, List, Literal, Optional

# Kubernetes problem
class K8SProblem(TypedDict):
    """Represents a Kubernetes problem."""
    type: Literal["symptom", "root_cause"]
    resource: str
    namespace: str
    issue: str
    description: str
    fix_suggestion: str

# Agent state
class AgentState(TypedDict):
    status: Literal["investigating", "found_issue", "completed"] # Used to manage the agent's flow
    problems_found: List[K8SProblem]
    suggestions: List[str]
    investigation_steps: List[str]

## Investigation steps

1. **Discover Phase**: Inspect the K8s cluster, find all problems across namespaces using K8S MCP server
2. **Filter Phase**: Identify which problems are symptoms vs root causes  
3. **Focus Phase**: Deep-dive into the main problematic resource using K8S MCP server
4. **Diagnose Phase**: Determine the single root cause of the issue
5. **Report Phase**: Output final diagnosis with clear fix for THE problem

In [ ]:
def initialize_agent_state() -> AgentState:
    return {
        "status": "investigating",
        "problems_found": [],
        "suggestions": [],
        "investigation_steps": []
    }